In [1]:
import os
import datasets
import json
from tqdm import tqdm
import jsonlines
import networkx as nx
import random

In [7]:
file_paths_tem2 = ['../../{}/seed_qa/1hop_{}_beforegen.jsonl',
                 '../../{}/seed_qa/2hop_{}_beforegen.jsonl']
file_paths_tem1 = ['../../QiJiaoSheng/genqa_from_seed/1hop_qa.jsonl',
                 '../../{}/genqa_from_seed/2hop_qa.jsonl',
                 '../../{}/genqa_from_seed/2hop_muit_alone.jsonl',
                 '../../{}/genqa_from_seed/1hop_alone.jsonl',
                 '../../{}/genqa_from_seed/1hop_muit.jsonl']
formula_list = [['QiJiaoSheng','qsc'],['SheXiangBao','sbw'],['SiMiaoFang','smf'],['XiaYuXue','xyx']]

In [20]:
file_paths = set()
for tem in file_paths_tem2:
    for form in formula_list:
        file_paths.add(tem.format(*form))
for tem in file_paths_tem1:
    for form in formula_list:
        file_paths.add(tem.format(form[0]))

In [22]:
file_paths = ['../../QiJiaoSheng/genqa_from_seed/1hop_qa.jsonl',
 '../../QiJiaoSheng/genqa_from_seed/2hop_muit_alone.jsonl',
 '../../QiJiaoSheng/genqa_from_seed/2hop_qa.jsonl',
 '../../QiJiaoSheng/seed_qa/1hop_qsc_beforegen.jsonl',
 '../../QiJiaoSheng/seed_qa/2hop_qsc_beforegen.jsonl',
 '../../SheXiangBao/genqa_from_seed/1hop_alone.jsonl',
 '../../SheXiangBao/genqa_from_seed/1hop_muit.jsonl',
 '../../SheXiangBao/genqa_from_seed/2hop_muit_alone.jsonl',
 '../../SheXiangBao/genqa_from_seed/2hop_qa.jsonl',
 '../../SheXiangBao/seed_qa/1hop_sbw_beforegen.jsonl',
 '../../SheXiangBao/seed_qa/2hop_sbw_beforegen.jsonl',
 '../../SiMiaoFang/genqa_from_seed/1hop_alone.jsonl',
 '../../SiMiaoFang/genqa_from_seed/1hop_muit.jsonl',
 '../../SiMiaoFang/genqa_from_seed/2hop_muit_alone.jsonl',
 '../../SiMiaoFang/genqa_from_seed/2hop_qa.jsonl',
 '../../SiMiaoFang/seed_qa/1hop_smf_beforegen.jsonl',
 '../../SiMiaoFang/seed_qa/2hop_smf_beforegen.jsonl',
 '../../XiaYuXue/genqa_from_seed/1hop_alone.jsonl',
 '../../XiaYuXue/genqa_from_seed/1hop_muit.jsonl',
 '../../XiaYuXue/genqa_from_seed/2hop_muit_alone.jsonl',
 '../../XiaYuXue/genqa_from_seed/2hop_qa.jsonl',
 '../../XiaYuXue/seed_qa/1hop_xyx_beforegen.jsonl',
 '../../XiaYuXue/seed_qa/2hop_xyx_beforegen.jsonl']

In [2]:
def get_head_tail(ans):
    
    #获取一系列三元组中的头节点集与尾结点集
    #输入为check_format好的一组三元组，三元组之间;隔开
    #返回head列表与tail列表
    
    objset = set()
    trilist = ans.split(';')
    htlist = []
    for tri in trilist:
        trii = tri[1:-1]
        #print(trii)
        triilist = trii.split(',')
        #print(triilist)
        objset.add(triilist[0])
        objset.add(triilist[2])
        htlist.append([triilist[0],triilist[2]])
    ishead = dict()
    istail = dict()
    for obj in objset:
        ishead[obj] = True
        istail[obj] = True
    for ht in htlist:
        ishead[ht[1]] = False
        istail[ht[0]] = False
    head = []
    tail = []
    for key in ishead:
        if ishead[key] == True:
            head.append(key)
    for key in istail:
        if istail[key] == True:
            tail.append(key)
    #print(tail)
    return head,tail

def get_each_hop_en(ans):
    head, tail = get_head_tail(ans)
    trilist = ans.split(';')
    htlist = []
    answer = []
    for tri in trilist:
        trii = tri[1:-1]
        #print(trii)
        triilist = trii.split(',')
        htlist.append([triilist[0],triilist[1],triilist[2]])
    cur_hop_list = head
    while len(cur_hop_list) != 0:
        nowlist = []
        for tris in htlist:
            if tris[0] in cur_hop_list:
                nowlist.append(tris[2])
        answer.append(cur_hop_list)    
        cur_hop_list = nowlist
        #print(cur_hop_list)
    return answer, htlist

def qa_isvalid_otherpro(qa_json):
    """
    合理性判断
    要求问题中只能有一条关系路径
    """
    each_hop, trilist = get_each_hop_en(qa_json['回答'])
    #print(each_hop, trilist)
    ques = qa_json['问题']
    each_hop_bool = []
    rel_list = []
    true_en = set()
    for hop_en in each_hop:
        for en in hop_en:
            for tris in trilist:
                if tris[0] == en:
                    rel_list.append(tris[1])
                    break
            break
    for hop_en in each_hop:
        cur_bool = []
        for en in hop_en:
            if en in ques:
                cur_bool.append(True)      
                true_en.add(en)
            else:
                cur_bool.append(False)
        each_hop_bool.append(cur_bool) 
    for formula in ['四妙方','麝香保心丸','下淤血汤','芪胶升白胶囊']:
        if formula in ques:
            true_en.add(formula)
    for idx, hop_bool in enumerate(each_hop_bool):
        each_hop_bool[idx] = all(hop_bool)
    #print(each_hop_bool)
    if any(each_hop_bool[:-1]) == False and each_hop_bool[-1] == True:
        qa_json['ques_en'] = each_hop[-1]
        qa_json['ans_en'] = each_hop[0]
        qa_json['rel_path'] = [rel_list]
        qa_json['entity'] = list(true_en)
        qa_json['distur'] = dict()
        for ent in qa_json['ques_en']:
            qa_json['distur'][ent] = qa_json['rel_path']
        qa_json['hop'] = len(qa_json['rel_path'][0])
        if len(each_hop) != qa_json['hop'] + 1:
            return False
        qa_json['回答'] = [qa_json['回答']]
        return qa_json
    if any(each_hop_bool[:-1]) == True:
        for idx in range(len(each_hop_bool)-2, -1, -1):
            if each_hop_bool[idx] == True:
                qa_json['ques_en'] = each_hop[idx]
                qa_json['ans_en'] = each_hop[-1]
                qa_json['rel_path'] = [rel_list[idx:]]
                qa_json['entity'] = list(true_en)
                qa_json['distur'] = dict()
                for ent in qa_json['ques_en']:
                    qa_json['distur'][ent] = qa_json['rel_path']
                qa_json['hop'] = len(qa_json['rel_path'][0])
                if len(each_hop) != qa_json['hop'] + 1:
                    return False
                if idx != 0:
                    new_trilist = []
                    no_en = []
                    for enslist in each_hop[0:idx]:
                        for ens in enslist:
                            no_en.append(ens)
                    for tris in trilist:
                        if tris[0] not in no_en:
                            new_trilist.append('({},{},{})'.format(tris[0], tris[1], tris[2]))
                    qa_json['回答'] = ';'.join(new_trilist)
                    qa_json['回答'] = [qa_json['回答']]
                else:
                    qa_json['回答'] = [qa_json['回答']]
                break
            else:
                continue
        return qa_json
    else:
        return False

In [3]:
def splithop(file_list, save_list):
    id1 = 0
    id2 = 0
    for file_path in file_list:
        cur_dataset = jsonlines.open(file_path)
        for cur_data in cur_dataset:
            cur_data['hop'] = len(cur_data['rel_path'][0])
            if cur_data['hop'] == 1:
                cur_data['id'] = id1
                id1 += 1
                save = open(save_list[0], 'a',encoding='utf-8')
                save.write(json.dumps(cur_data, ensure_ascii=False) + '\n')
                save.close()
            else:
                cur_data['id'] = id2
                id2 += 1
                save = open(save_list[1], 'a',encoding='utf-8')
                save.write(json.dumps(cur_data, ensure_ascii=False) + '\n')
                save.close()
        cur_dataset.close()   

In [25]:
splithop(file_paths, ['./1hop_qa.jsonl','./2hop_qa.jsonl'])

In [29]:
sum = 0
for file_path in file_paths:
    cur_dataset = jsonlines.open(file_path)
    i = 0
    for cur_data in cur_dataset:
        i+=1
    sum+=i
    print(i)
print(sum)

1167
239
67
137
8
389
863
344
89
131
10
725
1484
325
84
221
10
202
409
80
24
63
5
7076


In [4]:
def build_graph(graph: list) -> nx.Graph:
    G = nx.Graph()
    for triplet in graph:
        h, r, t = triplet
        G.add_edge(h, t, relation=r.strip())
    return G

In [5]:
def get_truth_paths(q_entity: list, a_entity: list, graph: nx.Graph) -> list:
    '''
    Get shortest paths connecting question and answer entities.
    '''
    # Select paths
    paths = []
    for h in q_entity:
        if h not in graph:
            continue
        for t in a_entity:
            if t not in graph:
                continue
            try:
                for p in nx.all_shortest_paths(graph, h, t):
                    paths.append(p)
            except:
                pass
    # Add relation to paths
    result_paths = []
    for p in paths:
        tmp = []
        for i in range(len(p)-1):
            u = p[i]
            v = p[i+1]
            tmp.append((u, graph[u][v]['relation'], v))
        result_paths.append(tmp)
    pathset = set()
    for cur in result_paths:
        cur_path = []
        for tri in cur:
            cur_path.append(tri[1])
        pathset.add('-'.join(cur_path))
    answer = []
    for singpath in pathset:
        answer.append(singpath.split('-'))
    return answer

In [33]:
g = [["四妙方", "草药", "黄柏"], ["四妙方", "草药", "苍术"], ["四妙方", "草药", "薏苡仁"], ["四妙方", "草药", "牛膝"], ["四妙方", "草药", "苍术"], ["四妙方", "疾病", "非酒精性脂肪性肝病"], ["四妙方", "症候", "痛风痹症"], ["四妙方", "症状", "足膝红肿疼痛"], ["四妙方", "症候", "湿热下注"], ["四妙方", "症候", "湿热痿证"], ["四妙方", "疾病", "清热燥湿"], ["黄柏", "效用", "清热燥湿"], ["牛膝", "效用", "肝肾"], ["牛膝", "效用", "强筋骨"], ["苍术", "效用", "燥湿健脾"], ["薏苡仁", "效用", "祛湿热"], ["薏苡仁", "效用", "利筋络"]]

In [34]:
graph = build_graph(g)

In [37]:
get_truth_paths(["四妙方"], ["肝肾", "利筋络", "清热燥湿", "非酒精性脂肪性肝病", "强筋骨", "痛风痹症", "足膝红肿疼痛", "湿热下注", "燥湿健脾", "祛湿热", "湿热痿证"], graph)

[['症候'], ['疾病'], ['症状'], ['草药', '效用']]

In [6]:
def split_from_raw(raw_file, save_list):
    id1 = 0
    id2 = 0
    cur_dataset = jsonlines.open(raw_file)
    for cur_data in cur_dataset:
        cur_g = build_graph(cur_data["graph"])
        rel_path = get_truth_paths(cur_data["q_entity"], cur_data["a_entity"], cur_g)
        if len(rel_path) == 1:
            if '效用' in rel_path[0]:
                newline = dict()
                tri_list = cur_data["graph"]
                new_list = []
                for tri in tri_list:
                    for idx, en in enumerate(tri):
                        ent = en.replace(';','；')
                        tri[idx] = ent.replace(',','，')
                    new_list.append('({},{},{})'.format(tri[0], tri[1], tri[2]))
                newline['问题'] = cur_data["question"]
                newline['回答']  = ';'.join(new_list)
                check = qa_isvalid_otherpro(newline)
                if check != False:
                    if check['hop'] == 1:
                        check['id'] = id1
                        id1 += 1
                        save = open(save_list[0], 'a',encoding='utf-8')
                        save.write(json.dumps(check, ensure_ascii=False) + '\n')
                        save.close()
                    else:
                        check['id'] = id2
                        id2 += 1
                        save = open(save_list[1], 'a',encoding='utf-8')
                        save.write(json.dumps(check, ensure_ascii=False) + '\n')
                        save.close()
    cur_dataset.close()   

In [11]:
split_from_raw('./raw_dataset.jsonl', ['./raw_1hop_qa.jsonl','./raw_2hop_qa.jsonl'])

In [7]:
def listequal(list1, list2):
    from collections import Counter
    are_lists_equal = (Counter(list1) == Counter(list2))
    return are_lists_equal

In [8]:
import random
def combine_1hop(qa_file, num, save_file):
    cur_dataset = jsonlines.open(qa_file)
    cur_dataset = list(cur_dataset)
    index = [i for i in range(len(cur_dataset))]
    sample_set = set()
    idx = 0
    while len(sample_set) < num:
        cur_sam = random.sample(index, 2)
        signal = '-'.join([str(cur_sam[0]),str(cur_sam[1])])
        if signal in sample_set:
            continue
        qa1 = cur_dataset[cur_sam[0]]
        qa2 = cur_dataset[cur_sam[1]]
        ans_en1 = ','.join(qa1["ans_en"])
        ans_en2 = ','.join(qa2["ans_en"])
        qa1_key = []
        qa2_key = []
        for key, value in qa1['distur'].items():
            qa1_key.append(key)
        for key, value in qa2['distur'].items():
            qa2_key.append(key)
        if listequal(qa1_key, qa2_key) and listequal(qa1['rel_path'][0], qa2['rel_path'][0]):
            continue
        if ans_en1 in ans_en2 or ans_en2 in ans_en1:
            continue
        sample_set.add(signal)
        new_distur = dict()
        for key, value in qa1['distur'].items():
            getval = new_distur.get(key, None)
            if getval == None:
                new_distur[key] = []
            for sin_path in value:
                if sin_path not in new_distur[key]:
                    new_distur[key].append(sin_path)
        for key, value in qa2['distur'].items():
            getval = new_distur.get(key, None)
            if getval == None:
                new_distur[key] = []
            for sin_path in value:
                if sin_path not in new_distur[key]:
                    new_distur[key].append(sin_path)
        while qa1['问题'][-1] == '；':
            qa1['问题'] = qa1['问题'][:-1]
        while qa2['问题'][-1] == '；':
            qa2['问题'] = qa2['问题'][:-1]   
        qa1['问题'] = qa1['问题'].replace('?','？')
        qa2['问题'] = qa2['问题'].replace('?','？')
        newline = dict()
        newline['问题'] = qa1['问题'] + qa2['问题']
        newline['回答'] = qa1['回答'] + qa2['回答']
        newline['ques_en'] = list(set(qa1['ques_en'] + qa2['ques_en']))
        newline['ans_en'] = list(set(qa1['ans_en'] + qa2['ans_en']))
        new_rel_path = []
        for p in qa1['rel_path']:
            if p not in new_rel_path:
                new_rel_path.append(p)
        for p in qa2['rel_path']:
            if p not in new_rel_path:
                new_rel_path.append(p)
        newline['rel_path'] = new_rel_path
        newline["entity"] = list(set(qa1['entity'] + qa2['entity']))
        newline["distur"] = new_distur
        newline['hop'] = max(qa1['hop'],qa2['hop'])
        newline['class'] = 'combine1_' + signal
        newline["strategy"]= "undetermined"
        newline['id'] = idx
        idx+=1
        save = open(save_file, 'a',encoding='utf-8')
        save.write(json.dumps(newline, ensure_ascii=False) + '\n')
        save.close()

In [32]:
combine_1hop('./1hop_qa.jsonl', 1182, './1hop_com.jsonl')

In [9]:
import random
def combine_2hop(qa_file1, qa_file2, num, save_file):
    cur_dataset1 = jsonlines.open(qa_file1)
    cur_dataset1 = list(cur_dataset1)
    index1 = [i for i in range(len(cur_dataset1))]
    cur_dataset2 = jsonlines.open(qa_file2)
    cur_dataset2 = list(cur_dataset2)
    index2 = [i for i in range(len(cur_dataset2))]
    sample_set = set()
    idx = 0
    while len(sample_set) < num:
        cur_sam1 = random.sample(index1, 1)
        cur_sam2 = random.sample(index2, 1)
        signal = '-'.join([str(cur_sam1[0]),str(cur_sam2[0])])
        if signal in sample_set:
            continue
        qa1 = cur_dataset1[cur_sam1[0]]
        qa2 = cur_dataset2[cur_sam2[0]]
        ans_en1 = ','.join(qa1["ans_en"])
        ans_en2 = ','.join(qa2["ans_en"])
        qa1_key = []
        qa2_key = []
        for key, value in qa1['distur'].items():
            qa1_key.append(key)
        for key, value in qa2['distur'].items():
            qa2_key.append(key)
        if listequal(qa1_key, qa2_key) and listequal(qa1['rel_path'][0], qa2['rel_path'][0]):
            continue
        if ans_en1 in ans_en2 or ans_en2 in ans_en1:
            continue
        sample_set.add(signal)
        new_distur = dict()
        for key, value in qa1['distur'].items():
            getval = new_distur.get(key, None)
            if getval == None:
                new_distur[key] = []
            for sin_path in value:
                if sin_path not in new_distur[key]:
                    new_distur[key].append(sin_path)
        for key, value in qa2['distur'].items():
            getval = new_distur.get(key, None)
            if getval == None:
                new_distur[key] = []
            for sin_path in value:
                if sin_path not in new_distur[key]:
                    new_distur[key].append(sin_path)
        while qa1['问题'][-1] == '；':
            qa1['问题'] = qa1['问题'][:-1]
        while qa2['问题'][-1] == '；':
            qa2['问题'] = qa2['问题'][:-1]   
        qa1['问题'] = qa1['问题'].replace('?','？')
        qa2['问题'] = qa2['问题'].replace('?','？')
        newline = dict()
        newline['问题'] = qa1['问题'] + qa2['问题']
        newline['回答'] = qa1['回答'] + qa2['回答']
        newline['ques_en'] = list(set(qa1['ques_en'] + qa2['ques_en']))
        newline['ans_en'] = list(set(qa1['ans_en'] + qa2['ans_en']))
        new_rel_path = []
        for p in qa1['rel_path']:
            if p not in new_rel_path:
                new_rel_path.append(p)
        for p in qa2['rel_path']:
            if p not in new_rel_path:
                new_rel_path.append(p)
        newline['rel_path'] = new_rel_path
        newline["entity"] = list(set(qa1['entity'] + qa2['entity']))
        newline["distur"] = new_distur
        newline['hop'] = max(qa1['hop'],qa2['hop'])
        newline['class'] = 'combine1_' + signal
        newline["strategy"]= "undetermined"
        newline['id'] = idx
        idx+=1
        save = open(save_file, 'a',encoding='utf-8')
        save.write(json.dumps(newline, ensure_ascii=False) + '\n')
        save.close()

In [34]:
combine_2hop('./1hop_qa.jsonl', './2hop_qa.jsonl', 1181, './2hop_com.jsonl')

In [12]:
want_num = 464
idx = 7036
cur_dataset = jsonlines.open('./raw_1hop_qa.jsonl')
cur_dataset = list(cur_dataset)
sam = random.sample(cur_dataset, want_num)
save = open('./1hop_qa.jsonl', 'a',encoding='utf-8')
for sam_s in sam:
    sam_s['class'] = 'samfromraw_' + str(sam_s['id'])
    sam_s['id'] = idx
    idx += 1
    sam_s["strategy"] = 'undetermined'
    save.write(json.dumps(sam_s, ensure_ascii=False) + '\n')
save.close()

In [13]:
idx = 40
cur_dataset = jsonlines.open('./raw_2hop_qa.jsonl')
cur_dataset = list(cur_dataset)
save = open('./2hop_qa.jsonl', 'a',encoding='utf-8')
for sam_s in cur_dataset:
    sam_s['class'] = 'samfromraw_' + str(sam_s['id'])
    sam_s['id'] = idx
    idx += 1
    sam_s["strategy"] = 'undetermined'
    save.write(json.dumps(sam_s, ensure_ascii=False) + '\n')
save.close()

In [14]:
['a'] in [['a'],['b']]

True

In [16]:
list(set(['a','a']))

['a']

In [2]:
def final_check_shuffle_split(file_pathlist, split_, train_file, test_file):
    all_data = []
    for file_path in file_pathlist:
        cur_dataset = jsonlines.open(file_path)
        for cur_data in cur_dataset:
            cur_data["entity"] = list(set(cur_data["entity"]))
            while cur_data['问题'][-1] == '；':
                cur_data['问题'] = cur_data['问题'][:-1]   
            all_data.append(cur_data)
    random.shuffle(all_data)
    train_data = all_data[0:int(len(all_data)*split_)]
    test_data = all_data[int(len(all_data)*split_):]
    id1 = 0
    id2 = 0
    for tr in train_data:
        tr['id'] = id1
        id1+=1
        save = open(train_file, 'a',encoding='utf-8')
        save.write(json.dumps(tr, ensure_ascii=False) + '\n')
        save.close()
    for te in test_data:
        te['id'] = id2
        id2+=1
        save = open(test_file, 'a',encoding='utf-8')
        save.write(json.dumps(te, ensure_ascii=False) + '\n')
        save.close()

In [6]:
gen_relpath_prompt = "你是一个中药复方知识图谱问答机器人，请生成一个或多个有效的关系路径，每个关系必须是[草药、效用、疾病、症候、症状、靶点、通路、成分、效应物]中的一个，该路径有助于回答以下问题：\n"
SOP = "<关系路径>{}</关系路径>"
SEP = "<分隔>"
ent_reg_prompt = "你是一个中药复方知识图谱问答机器人，请从输入的问题中提取有效的中药复方相关实体，该实体必须出现在输入中。\n"
SOPREG = "<实体>{}</实体>"
distur_prompt = "你是一个中药复方知识图谱问答机器人，对于输入的问题、实体、关系路径，请根据问题的语义，将关系路径分配给合适的实体，使得在知识图谱中，这些实体通过关系路径，可以搜索到问题的答案。\n"
SOPDIS = "<{}>{}</{}>"

In [7]:
def rel_2_token(rela_list):
    rel_list = []
    for rela in rela_list:
        rel_list.append(SOP.format(SEP.join(rela)))
    return ';'.join(rel_list)

def ent_2_token(enti_list):
    ent_list = []
    for ent in enti_list:
        ent_list.append(SOPREG.format(ent))
    return ';'.join(ent_list)

def distur_2_token(dis_dict):
    distur_list = []
    for key, path in dis_dict.items():
        rel_list_key = []
        for rela in path:
            rel_list_key.append(SOP.format(SEP.join(rela)))
        distur_list.append(SOPDIS.format(key, ';'.join(rel_list_key), key))
    return ';'.join(distur_list)

In [4]:
def gen_fine_form_chatglm(train_data, is_shuffle, save_file):
    output_list = []
    cur_dataset = jsonlines.open(train_data)
    for data in cur_dataset:
        gen_rel = dict()
        gen_rel['context'] = gen_relpath_prompt + data['问题']
        rel_list = []
        for rela in data['rel_path']:
            rel_list.append(SOP.format(SEP.join(rela)))
        gen_rel['target'] = ';'.join(rel_list)
        ent_reg = dict()
        ent_reg['context'] = ent_reg_prompt + data['问题']
        ent_list = []
        for ent in data['entity']:
            ent_list.append(SOPREG.format(ent))
        ent_reg['target'] = ';'.join(ent_list)
        distur = dict()
        distur['context'] = distur_prompt + '问题：' + data['问题'] + '\n' + '实体：' + ';'.join(data['entity']) + '\n' + '关系路径：' + ';'.join(rel_list)
        distur_list = []
        for key, path in data['distur'].items():
            rel_list_key = []
            for rela in path:
                rel_list_key.append(SOP.format(SEP.join(rela)))
            distur_list.append(SOPDIS.format(key, ';'.join(rel_list_key), key))
        distur['target'] = ';'.join(distur_list)
        output_list.append(gen_rel)
        output_list.append(ent_reg)
        output_list.append(distur)
    if is_shuffle:
        random.shuffle(output_list)
    for data in output_list:
        save = open(save_file, 'a',encoding='utf-8')
        data_json = json.dumps(data,ensure_ascii=False)
        save.write('{}\n'.format(data_json))
        save.close()

In [50]:
gen_fine_form_chatglm('./traindata.jsonl', True, './chatglm_fine.txt')

In [8]:
def token_2_rel(output):
    rel_list = []
    all_rel = ['草药','效用','疾病','症候','症状','靶点','通路','成分','效应物']
    pattern = r"<关系路径>(.*?)</关系路径>"
    for tokenstr in output:
        matched_items = re.findall(pattern, tokenstr)
        for items in matched_items:
            items_list = items.split('<分隔>')
            #print(items_list)
            isvaild = True
            for item in items_list:
                if item not in all_rel:
                    isvaild = False
            if isvaild == True:
                if items_list not in rel_list:
                    rel_list.append(items_list)
    return rel_list         

def token_2_ent(output):
    ent_list = set()
    pattern = r"<实体>(.*?)</实体>"
    matched_items = re.findall(pattern, output)
    for en in matched_items:
        ent_list.add(en)
    return list(ent_list)

def token_2_distur(output, ent_list):
    distur = dict()
    for entity in ent_list:
        distur[entity] = []
        pattern = r"<{}>(.*?)</{}>".format(entity,entity)
        matched_items = re.findall(pattern, output)
        rel_list = token_2_rel([output])
        for rel in rel_list:
            distur[entity].append(rel)
    return distur

In [ ]:
def gen_test_form_chatglm(test_data, save_file):
    

In [42]:
import re

# 定义正则表达式，用于匹配关系路径标签内的内容
pattern = r"<{}>(.*?)</{}>".format('关系路径','关系路径')

# 给定的字符串，包含多个关系路径标签
text = "<关系路径>靶点</关系路径><关系路径>草药<分隔>效用</关系路径></"

# 使用re.findall方法查找所有匹配的内容
matched_items = re.findall(pattern, text)

# 输出匹配结果
print(matched_items)

['靶点', '草药<分隔>效用']


In [9]:
def get_glm4_form(system, user, ass):
    gen_rel = dict()
    gen_rel['messages'] = []
    rel_sys = dict()
    rel_sys['content'] = system
    rel_sys['role'] = 'system'
    rel_user = dict()
    rel_user['content'] = user
    rel_user['role'] = 'user'
    rel_ass = dict()
    rel_ass['content'] = ass
    rel_ass['role'] = "assistant"
    gen_rel['messages'].append(rel_sys)
    gen_rel['messages'].append(rel_user)
    gen_rel['messages'].append(rel_ass)
    return gen_rel

In [10]:
def gen_fine_form_glm4(train_data, is_shuffle, save_file):
    output_list = []
    cur_dataset = jsonlines.open(train_data)
    for data in cur_dataset:
        relpath = rel_2_token(data['rel_path'])
        gen_rel = get_glm4_form(gen_relpath_prompt, data['问题'], rel_2_token(data['rel_path']))
        ent_reg = get_glm4_form(ent_reg_prompt, data['问题'], ent_2_token(data['entity']))
        distur_user = '问题：' + data['问题'] + '\n' + '实体：' + ';'.join(data['entity']) + '\n' + '关系路径：' + relpath
        distur = get_glm4_form(distur_prompt, distur_user, distur_2_token(data['distur']))
        output_list.append(gen_rel)
        output_list.append(ent_reg)
        output_list.append(distur)
    if is_shuffle:
        random.shuffle(output_list)
    for data in output_list:
        save = open(save_file, 'a',encoding='utf-8')
        save.write(json.dumps(data, ensure_ascii=False) + '\n')
        save.close()

In [11]:
def gen_fine_form_glm4_arr(train_data, is_shuffle, arr_list, save_file):
    output_list = []
    cur_dataset = jsonlines.open(train_data)
    for data in cur_dataset:
        relpath = rel_2_token(data['rel_path'])
        gen_rel = get_glm4_form(gen_relpath_prompt, data['问题'], rel_2_token(data['rel_path']))
        ent_reg = get_glm4_form(ent_reg_prompt, data['问题'], ent_2_token(data['entity']))
        distur_user = '问题：' + data['问题'] + '\n' + '实体：' + ';'.join(data['entity']) + '\n' + '关系路径：' + relpath
        distur = get_glm4_form(distur_prompt, distur_user, distur_2_token(data['distur']))
        if arr_list[0] == 1: 
            output_list.append(gen_rel)
        if arr_list[1] == 1:
            output_list.append(ent_reg)
        if arr_list[2] == 1:
            output_list.append(distur)
    if is_shuffle:
        random.shuffle(output_list)
    for data in output_list:
        save = open(save_file, 'a',encoding='utf-8')
        save.write(json.dumps(data, ensure_ascii=False) + '\n')
        save.close()

In [86]:
gen_fine_form_glm4('./traindata.jsonl', True, './glm4/glm4_fine.jsonl')

In [5]:
def get_llama3_form(instruction, inputs, output):
    gen_rel = dict()
    gen_rel['instruction'] = instruction
    gen_rel['input'] = inputs
    gen_rel['output'] = output
    return gen_rel

In [6]:
def gen_fine_form_llama3(train_data, is_shuffle, save_file):
    output_list = []
    cur_dataset = jsonlines.open(train_data)
    for data in cur_dataset:
        relpath = rel_2_token(data['rel_path'])
        gen_rel = get_llama3_form(gen_relpath_prompt, data['问题'], rel_2_token(data['rel_path']))
        ent_reg = get_llama3_form(ent_reg_prompt, data['问题'], ent_2_token(data['entity']))
        distur_user = '问题：' + data['问题'] + '\n' + '实体：' + ';'.join(data['entity']) + '\n' + '关系路径：' + relpath
        distur = get_llama3_form(distur_prompt, distur_user, distur_2_token(data['distur']))
        output_list.append(gen_rel)
        output_list.append(ent_reg)
        output_list.append(distur)
    if is_shuffle:
        random.shuffle(output_list)
    save = open(save_file, 'a',encoding='utf-8')
    save.write(json.dumps(output_list, ensure_ascii=False))
    save.close()

In [26]:
gen_fine_form_llama3('./traindata.jsonl', True, './llama3/llama3_fine.json')

In [ ]:
'<l(+)-精氨酸>(.*?)</l(+)-精氨酸>' '<l(+)-精氨酸><关系路径>草药</关系路径></l(+)-精氨酸>'

In [11]:
import re
matched_items = re.findall(r'<l(+)-精氨酸>(.*?)</l(+)-精氨酸>', '<l(+)-精氨酸><关系路径>草药</关系路径></l(+)-精氨酸>')

error: nothing to repeat at position 3

In [10]:
matched_items

[]

## 修正

In [29]:
import re
 
def find_all_occurrences(pattern, string):
    matches = re.finditer(pattern, string)
    start_index = [match.start() for match in matches]
    answer = [[start,start+len(pattern)-1] for start in start_index]
    return answer
 
# 示例使用
pattern = "麝香"
string = "麝香保心丸中的麝香"
indices = find_all_occurrences(pattern, string)
print(indices)

def search_decide(bigindex, smallindex):
    isalone = False
    for curindex in smallindex:
        cur_bool = []
        for curbigindex in bigindex:
            if curindex[0] >= curbigindex[0] and curindex[1] <= curbigindex[1]:
                cur_bool.append(False)
            else: 
                cur_bool.append(True)
        isalone = all(cur_bool)
        if isalone == True:
            return isalone
        else:
            continue
    return isalone

[[0, 1], [7, 8]]


In [31]:
search_decide([[0,8]], [[0, 1], [7, 8]])

False

In [7]:
for i in range(3,3):
    print(i)

In [32]:
cur_dataset = jsonlines.open('./traindata.jsonl')
new_data = []
for data in cur_dataset:
    repeatdic = dict()
    cur_len = len(data['entity'])
    new_ent = data['entity'].copy()
    for i in range(cur_len):
        for j in range(i+1, cur_len):
            if data['entity'][i] in data['entity'][j]:
                repeatdic[data['entity'][j]] = data['entity'][i]   
    for bigent, smallent in repeatdic.items():
        bigsearch = find_all_occurrences(bigent, data['问题'])
        smallsearch = find_all_occurrences(smallent, data['问题'])
        if not search_decide(bigsearch, smallsearch):
            new_ent.remove(smallent)
            #print(len(data['entity']))
    if len(new_ent) != len(data['entity']):
        print(data['id'], data['entity']) 
        data['entity'] = new_ent
        print(data['id'], data['entity']) 
    new_data.append(data)
for data in new_data:
    save = open('./traindata_new.jsonl', 'a',encoding='utf-8')
    save.write(json.dumps(data, ensure_ascii=False) + '\n')
    save.close()

31 ['麝香', '麝香保心丸']
31 ['麝香保心丸']
60 ['麝香', '麝香保心丸']
60 ['麝香保心丸']
86 ['麝香', '麝香保心丸']
86 ['麝香保心丸']
125 ['麝香', '麝香保心丸']
125 ['麝香保心丸']
183 ['麝香', '麝香保心丸']
183 ['麝香保心丸']
215 ['arid5b', '麝香', '麝香保心丸']
215 ['arid5b', '麝香保心丸']
308 ['麝香', '麝香保心丸']
308 ['麝香保心丸']
325 ['BDPLT1', '麝香', '麝香保心丸']
325 ['BDPLT1', '麝香保心丸']
329 ['麝香', '麝香保心丸']
329 ['麝香保心丸']
356 ['麝香', '麝香保心丸']
356 ['麝香保心丸']
358 ['下瘀血汤', '麝香', '麝香保心丸']
358 ['下瘀血汤', '麝香保心丸']
365 ['麝香', '麝香保心丸']
365 ['麝香保心丸']
402 ['麝香', '麝香保心丸']
402 ['麝香保心丸']
414 ['麝香', '麝香保心丸']
414 ['麝香保心丸']
449 ['麝香', '麝香保心丸']
449 ['麝香保心丸']
501 ['麝香', '麝香保心丸', '心脏保护']
501 ['麝香保心丸', '心脏保护']
516 ['麝香', '麝香保心丸']
516 ['麝香保心丸']
542 ['抑制肝星状细胞增殖活力', '下瘀血汤', '麝香', '麝香保心丸']
542 ['抑制肝星状细胞增殖活力', '下瘀血汤', '麝香保心丸']
580 ['麝香', '麝香保心丸']
580 ['麝香保心丸']
599 ['麝香', '麝香保心丸']
599 ['麝香保心丸']
671 ['苍术苷a', '麝香', '麝香保心丸']
671 ['苍术苷a', '麝香保心丸']
698 ['麝香', '麝香保心丸']
698 ['麝香保心丸']
715 ['下瘀血汤', '麝香', '麝香保心丸']
715 ['下瘀血汤', '麝香保心丸']
810 ['麝香', '麝香保心丸']
810 ['麝香保心丸']
835 ['麝香', '麝香保心丸']
835 ['麝香保心丸']
847 ['

In [18]:
a = [1,2]
b=a
b[0] = 3
a

[3, 2]

In [33]:
final_check_shuffle_split(['./traindata_new.jsonl'], 0.2, './20%traindata.jsonl', './20%testdata.jsonl')

In [34]:
final_check_shuffle_split(['./traindata_new.jsonl'], 0.4, './40%traindata.jsonl', './40%testdata.jsonl')

In [35]:
final_check_shuffle_split(['./traindata_new.jsonl'], 0.6, './60%traindata.jsonl', './60%testdata.jsonl')

In [36]:
final_check_shuffle_split(['./traindata_new.jsonl'], 0.8, './80%traindata.jsonl', './80%testdata.jsonl')

In [3]:
final_check_shuffle_split(['./traindata_new.jsonl'], 0.05, './5%traindata.jsonl', './5%testdata.jsonl')

In [40]:
gen_fine_form_chatglm('./20%traindata.jsonl', True, './chatglm_20%fine.txt')
gen_fine_form_chatglm('./40%traindata.jsonl', True, './chatglm_40%fine.txt')
gen_fine_form_chatglm('./60%traindata.jsonl', True, './chatglm_60%fine.txt')
gen_fine_form_chatglm('./80%traindata.jsonl', True, './chatglm_80%fine.txt')

In [5]:
gen_fine_form_chatglm('./5%traindata.jsonl', True, './chatglm_5%fine.txt')

In [46]:
gen_fine_form_glm4('./20%traindata.jsonl', True, './glm4/glm4_20%fine.jsonl')
gen_fine_form_glm4('./40%traindata.jsonl', True, './glm4/glm4_40%fine.jsonl')
gen_fine_form_glm4('./60%traindata.jsonl', True, './glm4/glm4_60%fine.jsonl')
gen_fine_form_glm4('./80%traindata.jsonl', True, './glm4/glm4_80%fine.jsonl')

In [10]:
gen_fine_form_glm4('./5%traindata.jsonl', True, './glm4/glm4_5%fine.jsonl')

In [49]:
gen_fine_form_llama3('./20%traindata.jsonl', True, './llama3/llama3_20%fine.json')
gen_fine_form_llama3('./40%traindata.jsonl', True, './llama3/llama3_40%fine.json')
gen_fine_form_llama3('./60%traindata.jsonl', True, './llama3/llama3_60%fine.json')
gen_fine_form_llama3('./80%traindata.jsonl', True, './llama3/llama3_80%fine.json')

In [7]:
gen_fine_form_llama3('./5%traindata.jsonl', True, './llama3/llama3_5%fine.json')

In [20]:
gen_fine_form_glm4_arr('./traindata_new.jsonl', True, [1,0,0], './glm4/glm4_fine_rel.jsonl')

In [21]:
gen_fine_form_glm4_arr('./traindata_new.jsonl', True, [0,1,0], './glm4/glm4_fine_ent.jsonl')

In [22]:
gen_fine_form_glm4_arr('./traindata_new.jsonl', True, [0,0,1], './glm4/glm4_fine_dis.jsonl')

In [23]:
gen_fine_form_glm4_arr('./traindata_new.jsonl', True, [1,0,1], './glm4/glm4_fine_rel_dis.jsonl')
gen_fine_form_glm4_arr('./traindata_new.jsonl', True, [0,1,1], './glm4/glm4_fine_ent_dis.jsonl')
gen_fine_form_glm4_arr('./traindata_new.jsonl', True, [1,1,0], './glm4/glm4_fine_rel_ent.jsonl')

## RAG

In [5]:
data = jsonlines.open('./traindata_new.jsonl')
answer = []
for d in data:
    if '麝香' in d['问题'] or '冠心病' in d['问题']:
        continue
    answer.append(d)
for d in answer:
    save = open('./traindata_rag.jsonl', 'a',encoding='utf-8')
    save.write(json.dumps(d, ensure_ascii=False) + '\n')
    save.close()

In [12]:
gen_fine_form_glm4('./traindata_rag.jsonl', True, './glm4/glm4_ragfine.jsonl')

In [18]:
data = jsonlines.open('./traindata_new.jsonl')
answer = []
dis_set = set()
for d in data:
    if '麝香' in d['问题'] or '冠心病' in d['问题']:
        if 'combine' not in d['class']:
            if '麝香保心丸' in list(d['distur'].keys())[0]:
                key = list(d['distur'].keys())[0]
                value = '-'.join(d['distur'][key][0])
                signal = key + '-' + value
                if signal not in dis_set:
                    dis_set.add(signal)
                    d['rag'] = ''
                    answer.append(d)
for d in answer:
    save = open('./testdata_rag.jsonl', 'a',encoding='utf-8')
    save.write(json.dumps(d, ensure_ascii=False) + '\n')
    save.close()